In [6]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [8]:
subject = 'Bahia - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [9]:
data = pd.read_csv('2003_mo_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego,Bahia - Consumo de Cimento (t)
0,2003-1,0.299858,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.669899,39.798880,1.317344e+08,8.384593e+06,8.566149,1.216359e+08,8.348779,151.297
1,2003-2,0.301903,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.670210,39.480034,1.318964e+08,8.391946e+06,8.569210,1.216914e+08,8.342979,138.707
2,2003-3,0.303709,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.670521,39.400256,1.320584e+08,8.399299e+06,8.572270,1.217469e+08,8.337179,135.009
3,2003-4,0.305311,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.670831,39.417185,1.322204e+08,8.406652e+06,8.575331,1.218023e+08,8.331379,126.554
4,2003-5,0.306860,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.671142,39.479943,1.323824e+08,8.414005e+06,8.578392,1.218578e+08,8.325579,137.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.597113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.069163,NaN,NaN,NaN,NaN,NaN,366.305
236,2022-9,0.596178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.752943,NaN,NaN,NaN,NaN,NaN,346.042
237,2022-10,0.594662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.537361,NaN,NaN,NaN,NaN,NaN,347.901
238,2022-11,0.592436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.971241,NaN,NaN,NaN,NaN,NaN,310.845


In [10]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.440329,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.143224,-1.723080,-1.703176,-0.816345,-2.235724,-2.158201,-0.884052
1,-1.404914,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.108465,-1.735985,-1.684201,-0.772933,-2.194204,-2.117854,-0.885724
2,-1.373637,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.073707,-1.739214,-1.665225,-0.729522,-2.152684,-2.077508,-0.887395
3,-1.345888,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.038948,-1.738529,-1.646249,-0.686111,-2.111164,-2.037161,-0.889067
4,-1.319065,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.004190,-1.735989,-1.627274,-0.642700,-2.069645,-1.996814,-0.890739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.796141,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.261516,0.511535,1.096947,-1.668064,0.877256,0.724962,1.200853
188,1.811284,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.253736,0.513264,1.085441,-1.663847,0.851021,0.704849,1.200487
189,1.833479,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.245956,0.526782,1.073935,-1.659631,0.824786,0.684737,1.200122
190,1.853109,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.238176,0.525379,1.062429,-1.655414,0.798552,0.664624,1.199756


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      129.857
1      126.813
2      152.225
3      136.288
4      148.117
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Bahia - Consumo de Cimento (t), Length: 240, dtype: float64

In [12]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.440329,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.143224,-1.723080,-1.703176,-0.816345,-2.235724,-2.158201,-0.884052
1,-1.404914,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.108465,-1.735985,-1.684201,-0.772933,-2.194204,-2.117854,-0.885724
2,-1.373637,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.073707,-1.739214,-1.665225,-0.729522,-2.152684,-2.077508,-0.887395
3,-1.345888,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.038948,-1.738529,-1.646249,-0.686111,-2.111164,-2.037161,-0.889067
4,-1.319065,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.004190,-1.735989,-1.627274,-0.642700,-2.069645,-1.996814,-0.890739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.097864,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.430739,0.617097,1.205171,-1.332406,1.004714,1.051323,1.273469
158,1.120412,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.428086,0.590103,1.206522,-1.357999,1.007770,1.045502,1.268173
159,1.139591,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.425433,0.561454,1.207873,-1.383592,1.010826,1.039682,1.262877
160,1.156903,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.422780,0.523492,1.209225,-1.409185,1.013881,1.033862,1.257581


In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      129.857
1      126.813
2      152.225
3      136.288
4      148.117
        ...   
157    213.266
158    285.938
159    219.576
160    267.203
161    240.714
Name: Bahia - Consumo de Cimento (t), Length: 162, dtype: float64

In [14]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
162,1.191770,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.417474,0.500213,1.211927,-1.460371,1.019993,1.022222,1.246990
163,1.216939,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.414821,0.496834,1.213278,-1.485964,1.023048,1.016401,1.241694
164,1.243300,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.412168,0.484739,1.214629,-1.511558,1.026104,1.010581,1.236398
165,1.273404,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.409516,0.482071,1.215980,-1.537151,1.029160,1.004761,1.231103
166,1.301186,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.406863,0.473941,1.217331,-1.562744,1.032215,0.998941,1.225807
167,1.333756,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.404210,0.434848,1.218682,-1.588337,1.035271,0.993120,1.220511
168,1.365896,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.401557,0.435705,1.220034,-1.613930,1.038327,0.987300,1.215215
169,1.395788,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.394425,0.428428,1.216488,-1.620901,1.040208,0.977171,1.214232
170,1.426027,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.387293,0.426831,1.212943,-1.627872,1.042089,0.967042,1.213248
171,1.463270,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.380162,0.407983,1.209397,-1.634843,1.043970,0.956913,1.212264


In [15]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    250.101
163    277.528
164    270.092
165    278.146
166    257.458
167    231.748
168    268.336
169    223.453
170    241.464
171    238.901
172    191.989
173    272.452
174    261.009
175    292.688
176    258.881
177    276.879
178    255.774
179    208.326
180    291.428
181    249.430
182    241.612
183    252.303
184    281.912
185    200.213
186    270.511
187    281.466
188    285.535
189    328.259
190    298.078
191    264.838
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [16]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [17]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
#     train, train_val = validation_splitter(train_input, 6)
#     target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [20]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [21]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3087440751, 3666903063, 2552058620, 1216835641, 929956629, 1223562457, 4179932844, 1853179175, 2395913995, 1375589850, 869057096, 3620091391, 834038104, 1019828791, 1587637726, 416357323, 2058425316, 3837850274, 3650684689, 3787766180, 1092564551, 3102168795, 2739444285, 4062915347, 3087772582]


Step: 0 ___________________________________________
val_loss: 33596.68359375
winner_seed: 3087440751


Step: 1 ___________________________________________
val_loss: 47471.6328125


Step: 2 ___________________________________________
val_loss: 17224.587890625
winner_seed: 2552058620


Step: 3 ___________________________________________
val_loss: 48926.71484375


Step: 4 ___________________________________________
val_loss: 15094.115234375
winner_seed: 929956629


Step: 5 ___________________________________________
val_loss: 44672.9765625


Step: 6 ___________________________________________
val_loss: 36193.25390625


Step: 7 ___________________________________________
val_loss: 28547.390625




In [22]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 28ms/step - loss: 70762.7188 - val_loss: 46121.8008
Epoch 2/10000
5/5 [==============================] - 0s 7ms/step - loss: 68242.8281 - val_loss: 95961.1641
Epoch 3/10000
5/5 [==============================] - 0s 7ms/step - loss: 59072.4258 - val_loss: 6756.4531
Epoch 4/10000
5/5 [==============================] - 0s 7ms/step - loss: 34384.9609 - val_loss: 44410.6562
Epoch 5/10000
5/5 [==============================] - 0s 7ms/step - loss: 10281.8975 - val_loss: 60186.0391
Epoch 6/10000
5/5 [==============================] - 0s 8ms/step - loss: 1421.1132 - val_loss: 42904.5430
Epoch 7/10000
5/5 [==============================] - 0s 7ms/step - loss: 666.6014 - val_loss: 46883.9062
Epoch 8/10000
5/5 [==============================] - 0s 8ms/step - loss: 687.3931 - val_loss: 42943.7227
Epoch 9/10000
5/5 [==============================] - 0s 8ms/step - loss: 484.3922 - val_loss: 43323.3438
Epoch 10/10000
5/5 [========================

5/5 [==============================] - 0s 6ms/step - loss: 1510.1134 - val_loss: 96975.4453
Epoch 79/10000
5/5 [==============================] - 0s 12ms/step - loss: 1896.3263 - val_loss: 82857.5469
Epoch 80/10000
5/5 [==============================] - 0s 7ms/step - loss: 1985.5701 - val_loss: 101212.6953
Epoch 81/10000
5/5 [==============================] - 0s 7ms/step - loss: 2577.6733 - val_loss: 110327.7969
Epoch 82/10000
5/5 [==============================] - 0s 7ms/step - loss: 2454.4644 - val_loss: 101100.6484
Epoch 83/10000
5/5 [==============================] - 0s 7ms/step - loss: 2088.8042 - val_loss: 89751.5312
Epoch 84/10000
5/5 [==============================] - 0s 7ms/step - loss: 1916.1552 - val_loss: 109212.0547
Epoch 85/10000
5/5 [==============================] - 0s 7ms/step - loss: 1945.4108 - val_loss: 89018.7266
Epoch 86/10000
5/5 [==============================] - 0s 6ms/step - loss: 2179.8508 - val_loss: 108407.7734
Epoch 87/10000
5/5 [==========================

5/5 [==============================] - 0s 7ms/step - loss: 474.9884 - val_loss: 44659.4219
Epoch 155/10000
5/5 [==============================] - 0s 7ms/step - loss: 551.1142 - val_loss: 48855.2070
Epoch 156/10000
5/5 [==============================] - 0s 7ms/step - loss: 580.3477 - val_loss: 48995.8047
Epoch 157/10000
5/5 [==============================] - 0s 7ms/step - loss: 498.7071 - val_loss: 48354.2969
Epoch 158/10000
5/5 [==============================] - 0s 7ms/step - loss: 628.9984 - val_loss: 50176.6289
Epoch 159/10000
5/5 [==============================] - 0s 6ms/step - loss: 593.8937 - val_loss: 44674.0469
Epoch 160/10000
5/5 [==============================] - 0s 7ms/step - loss: 613.4746 - val_loss: 46238.7812
Epoch 161/10000
5/5 [==============================] - 0s 7ms/step - loss: 645.7176 - val_loss: 48555.9766
Epoch 162/10000
5/5 [==============================] - 0s 7ms/step - loss: 575.7727 - val_loss: 44770.0430
Epoch 163/10000
5/5 [==============================] 

5/5 [==============================] - 0s 7ms/step - loss: 751.0035 - val_loss: 34240.3359
Epoch 231/10000
5/5 [==============================] - 0s 7ms/step - loss: 502.9473 - val_loss: 32191.5508
Epoch 232/10000
5/5 [==============================] - 0s 7ms/step - loss: 647.2759 - val_loss: 50572.3125
Epoch 233/10000
5/5 [==============================] - 0s 7ms/step - loss: 1065.3741 - val_loss: 45578.4297
Epoch 234/10000
5/5 [==============================] - 0s 6ms/step - loss: 739.0164 - val_loss: 45059.6758
Epoch 235/10000
5/5 [==============================] - 0s 6ms/step - loss: 764.3983 - val_loss: 41380.9883
Epoch 236/10000
5/5 [==============================] - 0s 7ms/step - loss: 557.7856 - val_loss: 41827.1367
Epoch 237/10000
5/5 [==============================] - 0s 11ms/step - loss: 668.1111 - val_loss: 40999.6992
Epoch 238/10000
5/5 [==============================] - 0s 7ms/step - loss: 694.3544 - val_loss: 40450.5312
Epoch 239/10000
5/5 [==============================

5/5 [==============================] - 0s 6ms/step - loss: 642.7585 - val_loss: 48895.8672
Epoch 307/10000
5/5 [==============================] - 0s 6ms/step - loss: 974.8203 - val_loss: 59154.8359
Epoch 308/10000
5/5 [==============================] - 0s 6ms/step - loss: 840.3202 - val_loss: 49425.0586
Epoch 309/10000
5/5 [==============================] - 0s 6ms/step - loss: 944.4700 - val_loss: 57350.7188
Epoch 310/10000
5/5 [==============================] - 0s 7ms/step - loss: 855.0098 - val_loss: 53721.4922
Epoch 311/10000
5/5 [==============================] - 0s 7ms/step - loss: 733.2249 - val_loss: 60189.4492
Epoch 312/10000
5/5 [==============================] - 0s 7ms/step - loss: 694.0012 - val_loss: 54691.0195
Epoch 313/10000
5/5 [==============================] - 0s 6ms/step - loss: 646.1424 - val_loss: 49523.7148
Epoch 314/10000
5/5 [==============================] - 0s 7ms/step - loss: 674.5055 - val_loss: 50761.8008
Epoch 315/10000
5/5 [==============================] 

...

Epoch 1217/10000
5/5 [==============================] - 0s 6ms/step - loss: 669.9072 - val_loss: 21776.1387
Epoch 1218/10000
5/5 [==============================] - 0s 6ms/step - loss: 497.5457 - val_loss: 27139.6953
Epoch 1219/10000
5/5 [==============================] - 0s 6ms/step - loss: 484.6211 - val_loss: 27370.3770
Epoch 1220/10000
5/5 [==============================] - 0s 5ms/step - loss: 509.6815 - val_loss: 28700.6875
Epoch 1221/10000
5/5 [==============================] - 0s 5ms/step - loss: 456.0750 - val_loss: 31030.4082
Epoch 1222/10000
5/5 [==============================] - 0s 6ms/step - loss: 679.0002 - val_loss: 23807.8906
Epoch 1223/10000
5/5 [==============================] - 0s 6ms/step - loss: 547.5131 - val_loss: 31777.9121
Epoch 1224/10000
5/5 [==============================] - 0s 7ms/step - loss: 474.3041 - val_loss: 31058.6406
Epoch 1225/10000
5/5 [==============================] - 0s 6ms/step - loss: 486.9378 - val_loss: 31764.4004
Epoch 1226/10000
5/5 [======

Epoch 1293/10000
5/5 [==============================] - 0s 6ms/step - loss: 707.8546 - val_loss: 26983.5527
Epoch 1294/10000
5/5 [==============================] - 0s 6ms/step - loss: 665.3494 - val_loss: 27713.5801
Epoch 1295/10000
5/5 [==============================] - 0s 6ms/step - loss: 694.5486 - val_loss: 26556.1973
Epoch 1296/10000
5/5 [==============================] - 0s 6ms/step - loss: 638.8948 - val_loss: 24981.4375
Epoch 1297/10000
5/5 [==============================] - 0s 6ms/step - loss: 714.3447 - val_loss: 24435.6934
Epoch 1298/10000
5/5 [==============================] - 0s 7ms/step - loss: 646.4563 - val_loss: 23341.0273
Epoch 1299/10000
5/5 [==============================] - 0s 7ms/step - loss: 641.2899 - val_loss: 23150.3340
Epoch 1300/10000
5/5 [==============================] - 0s 6ms/step - loss: 662.5024 - val_loss: 24486.3633
Epoch 1301/10000
5/5 [==============================] - 0s 6ms/step - loss: 703.4500 - val_loss: 19198.0527
Epoch 1302/10000
5/5 [======

Epoch 1369/10000
5/5 [==============================] - 0s 6ms/step - loss: 1159.5173 - val_loss: 35898.4727
Epoch 1370/10000
5/5 [==============================] - 0s 6ms/step - loss: 1177.7231 - val_loss: 35447.8438
Epoch 1371/10000
5/5 [==============================] - 0s 7ms/step - loss: 1215.6058 - val_loss: 26445.8184
Epoch 1372/10000
5/5 [==============================] - 0s 6ms/step - loss: 953.1254 - val_loss: 25858.3105
Epoch 1373/10000
5/5 [==============================] - 0s 6ms/step - loss: 839.0909 - val_loss: 23975.3105
Epoch 1374/10000
5/5 [==============================] - 0s 6ms/step - loss: 848.3754 - val_loss: 24842.9766
Epoch 1375/10000
5/5 [==============================] - 0s 6ms/step - loss: 840.6279 - val_loss: 25618.4316
Epoch 1376/10000
5/5 [==============================] - 0s 6ms/step - loss: 822.3115 - val_loss: 26793.6602
Epoch 1377/10000
5/5 [==============================] - 0s 6ms/step - loss: 804.2288 - val_loss: 30392.2969
Epoch 1378/10000
5/5 [===

Epoch 1445/10000
5/5 [==============================] - 0s 6ms/step - loss: 596.4387 - val_loss: 31435.3633
Epoch 1446/10000
5/5 [==============================] - 0s 6ms/step - loss: 562.9383 - val_loss: 33111.1250
Epoch 1447/10000
5/5 [==============================] - 0s 6ms/step - loss: 604.1426 - val_loss: 32696.1289
Epoch 1448/10000
5/5 [==============================] - 0s 6ms/step - loss: 602.9065 - val_loss: 31091.4746
Epoch 1449/10000
5/5 [==============================] - 0s 6ms/step - loss: 674.7493 - val_loss: 32145.1074
Epoch 1450/10000
5/5 [==============================] - 0s 6ms/step - loss: 626.3325 - val_loss: 31397.6992
Epoch 1451/10000
5/5 [==============================] - 0s 6ms/step - loss: 614.9904 - val_loss: 30557.9824
Epoch 1452/10000
5/5 [==============================] - 0s 6ms/step - loss: 603.6635 - val_loss: 32463.8164
Epoch 1453/10000
5/5 [==============================] - 0s 6ms/step - loss: 592.7338 - val_loss: 32065.6426
Epoch 1454/10000
5/5 [======

Epoch 1521/10000
5/5 [==============================] - 0s 6ms/step - loss: 859.9147 - val_loss: 27437.6426
Epoch 1522/10000
5/5 [==============================] - 0s 6ms/step - loss: 799.2729 - val_loss: 27728.4707
Epoch 1523/10000
5/5 [==============================] - 0s 6ms/step - loss: 832.4785 - val_loss: 28213.1309
Epoch 1524/10000
5/5 [==============================] - 0s 6ms/step - loss: 834.8546 - val_loss: 28008.1133
Epoch 1525/10000
5/5 [==============================] - 0s 6ms/step - loss: 760.8140 - val_loss: 27386.8555
Epoch 1526/10000
5/5 [==============================] - 0s 6ms/step - loss: 726.2781 - val_loss: 26400.0547
Epoch 1527/10000
5/5 [==============================] - 0s 5ms/step - loss: 811.9570 - val_loss: 28332.8887
Epoch 1528/10000
5/5 [==============================] - 0s 5ms/step - loss: 798.7429 - val_loss: 29172.6641
Epoch 1529/10000
5/5 [==============================] - 0s 6ms/step - loss: 802.0542 - val_loss: 29372.6055
Epoch 1530/10000
5/5 [======

In [23]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [24]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702,137.138702
Target,250.101,277.528,270.092,278.146,257.458,231.748,268.336,223.453,241.464,238.901,191.989,272.452,261.009,292.688,258.881,276.879,255.774,208.326,291.428,249.43,241.612,252.303,281.912,200.213,270.511,281.466,285.535,328.259,298.078,264.838
Error,112.962296,140.389313,132.953308,141.007294,120.319305,94.609299,131.197296,86.314301,104.325302,101.762299,54.850296,135.313293,123.8703,155.549286,121.74231,139.740295,118.6353,71.187302,154.289307,112.29129,104.473297,115.164291,144.773285,63.074295,133.372284,144.327301,148.396301,191.1203,160.939301,127.69931


In [25]:
display(mae)
display(mape)

122.8883

0.46537665

In [26]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [27]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1645.6642]] - Target[3001.6679999999997]| =  Error: [[1356.0038]]; MAPE:[[0.4517501]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1645.6642]] - Target[3070.4549999999995]| =  Error: [[1424.7909]]; MAPE:[[0.46403247]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[822.83215]] - Target[1728.6870000000001]| =  Error: [[905.85486]]; MAPE:[[0.5240132]]


[array([[1356.0038]], dtype=float32),
 array([[1424.7909]], dtype=float32),
 array([[905.85486]], dtype=float32)]

1228.8832

0.47993192